In [1]:
import sys
import os
from tensorflow.keras.models import load_model
import numpy as np
from PIL import Image
from sklearn.metrics import accuracy_score
import tensorflow as tf

parent_directory = os.path.abspath('/Users/subhamgogoi/Coding/Jupyter/Projects/CIFAR10-CNN-Classifier')
if parent_directory not in sys.path:
    sys.path.append(parent_directory)

from image_preprocessing import preprocess_multiclass
from image_labelling import auto_label_images

mobileNetV2_model = load_model('mobileNetV2.h5')

2024-09-22 21:14:24.488735: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-09-22 21:14:24.488761: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-09-22 21:14:24.488767: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-09-22 21:14:24.488800: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-09-22 21:14:24.488817: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


### Prediction on a single image

In [2]:
image_path = '/Users/subhamgogoi/Pictures/testImages-multiClass/Airplane/Airplane_1.jpg'
img_name = os.path.basename(image_path)
image = Image.open(image_path)

image = image.resize((224,224))
image_array = np.array(image)
image_array = image_array / 255.0
image_array = np.reshape(image_array, (1,224,224,3))

prediction = mobileNetV2_model.predict(image_array)
predicted_class = np.argmax(prediction)
class_names = ['Airplane', 'Car', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']
class_index = {class_name:i for i, class_name in enumerate(class_names)}
print(f"Predicted class: {class_names[predicted_class]}")
# print(f'Class indices - {class_index}')

# Evaluate loss and accuracy
true_class_name = os.path.basename(os.path.dirname(image_path))
true_class_index = class_index[true_class_name]
true_class_index_hotencoded = tf.keras.utils.to_categorical(true_class_index, num_classes=10)
true_class_index_hotencoded = np.reshape(true_class_index_hotencoded, (1,10))

loss = tf.keras.losses.categorical_crossentropy(true_class_index_hotencoded, prediction)
print(f'Test loss on {img_name} - {loss[0]:.3f}')

accuracy = accuracy_score([true_class_index], [predicted_class])
print(f'Test accuracy on {img_name} - {accuracy}')

2024-09-22 21:14:26.482941: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 848ms/step
Predicted class: Airplane
Test loss on Airplane_1.jpg - 0.015
Test accuracy on Airplane_1.jpg - 1.0


### Prediction on a batch of images

In [3]:
# Preprocess the images
image_dir = '/Users/subhamgogoi/Pictures/testImages-multiClass'

image_batch, filenames_predictions = preprocess_multiclass(image_dir)
true_labels, filenames_original = auto_label_images(image_dir)

# Sort filenames and true_labels to match the order in preprocess_multiclass
filename_to_label = dict(zip(filenames_original, true_labels))
true_labels = [filename_to_label[fname] for fname in filenames_predictions]

# Make predictions on the batch
predictions = mobileNetV2_model.predict(image_batch)

# Define CIFAR-10 class names
class_names = ['Airplane', 'Car', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']

# Interpret and print predictions
predicted_classes = np.argmax(predictions, axis=1)
for pred_class, filename in zip(predicted_classes, filenames_predictions):
    print(f"{filename} is predicted as: {class_names[pred_class]}")

Populated 66 images
3/3 [==============================] - 1s 272ms/step
Airplane_1.jpg is predicted as: Airplane
Bird_1.jpg is predicted as: Bird
Bird_2.jpg is predicted as: Bird
Bird_3.jpg is predicted as: Bird
Bird_4.jpg is predicted as: Bird
Bird_5.jpg is predicted as: Bird
Bird_6.jpg is predicted as: Bird
Car_1.jpg is predicted as: Truck
Cat_1.jpg is predicted as: Cat
Cat_2.jpg is predicted as: Cat
airplane_2.jpg is predicted as: Airplane
airplane_3.jpg is predicted as: Airplane
airplane_4.jpg is predicted as: Airplane
airplane_5.jpg is predicted as: Airplane
airplane_6.jpg is predicted as: Airplane
airplane_7.jpg is predicted as: Airplane
airplane_8.jpg is predicted as: Airplane
car_2.jpg is predicted as: Car
car_3.jpg is predicted as: Car
car_4.jpg is predicted as: Car
car_5.jpg is predicted as: Car
car_6.jpg is predicted as: Car
car_7.jpg is predicted as: Car
cat_3.jpg is predicted as: Cat
cat_4.jpg is predicted as: Cat
cat_5.jpg is predicted as: Cat
cat_6.jpg is predicted as: 

### Evaluate loss and accuracy

In [4]:
true_labels_hot_encoded = tf.keras.utils.to_categorical(true_labels, num_classes=10)

In [7]:
loss = tf.keras.losses.categorical_crossentropy(true_labels_hot_encoded, predictions).numpy().mean()
print(f"Loss: {loss:.3f}")

test_acc = accuracy_score(true_labels, predicted_classes)
print(f"Test accuracy : {test_acc * 100 :.2f} %")

Loss: 0.312
Test accuracy : 93.94 %
